In [12]:
from string import punctuation
from nltk.corpus import stopwords
from collections import Counter
from scipy import sparse
import numpy as np
import pandas as pd

### 1. Leitura do texto central

In [13]:
inputFile = "data/HN_posts_year_to_Sep_26_2016.csv"
language = 'english'
df = pd.read_csv(inputFile)
corpus = df['title']

In [3]:
inputFile = "data/BrasCubas-Assis.txt"
language = 'portuguese'
with open(inputFile) as f:
    corpus = f.readlines()
    corpus = [paragraph.rstrip() for paragraph in corpus if paragraph != '\n']

### 2. Pré-processamento do texto

In [14]:
punctranslation = str.maketrans(dict.fromkeys(punctuation))
setStopwords = set(stopwords.words(language))

# Realiza a tokenização e tratamento dos parágrafos do texto
def tokenize(corpus: str) -> list:
    corpusTokenized = []
    for paragraph in corpus:
        paragraph = paragraph.lower()                                       # Tratamento de case-sensitive
        paragraph = paragraph.encode('utf8', 'ignore').decode()             # Eliminação de caracteres fora de UTF-8
        paragraph = paragraph.translate(punctranslation)                    # Eliminação de pontuações
        corpusTokenized.append([token for token in paragraph.split()        # Eliminação de stopwords
                                if token not in setStopwords])
    return corpusTokenized
    
corpusTokenized = tokenize(corpus)

### 3. Contagem de unigramas do texto

In [15]:
unigrams = Counter()
for paragraph in corpusTokenized:
    for token in paragraph:
        unigrams[token] += 1

# Mapeamento de acesso (Token <-> Índice representativo)
token2index = {token: index for index, token in enumerate(unigrams.keys())}
index2token = {index: token for token, index in token2index.items()}

### 4. Contagem de bigramas do texto

In [16]:
# Contador de bigramas, considerando uma determinada janela de contexto (nesse caso, 2
# palavras antes e duas palavras depois)
skipgrams = Counter()
gap = 3

for paragraph in corpusTokenized:
    tokens = [token2index[tok] for tok in paragraph]
    
    # Para cada palavra no parágrafo, realiza a análise dos contextos da vizinhança
    for indexWord, word in enumerate(paragraph):
        indexContextMin = max(0, indexWord - gap)
        indexContextMax = min(len(paragraph)-1, indexWord + gap)

        # Para cada contexto da vizinhança, crie um bigrama com a palavra central
        indexContexts = [index for index in range(indexContextMin, indexContextMax + 1) if index != indexWord]
        for indexContext in indexContexts:
            skipgram = (tokens[indexWord], tokens[indexContext])
            skipgrams[skipgram] += 1

# Exibição dos 10 bigramas mais comuns presentes no texto, considerando a janela de contexto estipulada
mostCommon = [(index2token[skipgram[0][0]], index2token[skipgram[0][1]], skipgram[1]) 
               for skipgram in skipgrams.most_common(10)]
mostCommon

[('show', 'hn', 10263),
 ('hn', 'show', 10263),
 ('ask', 'hn', 9255),
 ('hn', 'ask', 9255),
 ('open', 'source', 2036),
 ('source', 'open', 2036),
 ('machine', 'learning', 1211),
 ('learning', 'machine', 1211),
 ('silicon', 'valley', 1060),
 ('valley', 'silicon', 1060)]

### 5. Criação da matrix de frequência termo-a-termo

In [17]:
# Mapeamento das entradas da matriz esparça de frequência entre os bigramas do texto
rowsMatrix = []
columnsMatrix = []
dataMatrix = []

for (token1, token2), skipgramCount in skipgrams.items():
    rowsMatrix.append(token1)
    columnsMatrix.append(token2)
    dataMatrix.append(skipgramCount)

wwMatrix = sparse.csr_matrix((dataMatrix, (rowsMatrix, columnsMatrix)))
wwMatrix

<99366x99366 sparse matrix of type '<class 'numpy.int64'>'
	with 3792872 stored elements in Compressed Sparse Row format>

### 6. Criação da matriz PPMI

In [18]:
# Número total de bigramas presente na matriz de frequência
numSkipgrams = wwMatrix.sum()

# Mapeamento das entradas da matriz PPMI
rowsIndex = []
columnsIndex = []
ppmiData = []

# Vetor de frequência total de cada palavra em todos os possíveis contextos
sumWords = np.array(wwMatrix.sum(axis=0)).flatten()

# Vetor de frequência total de cada contexto para todas as possíveis palavras
sumContexts = np.array(wwMatrix.sum(axis=1)).flatten()

for (tokenWord, tokenContext), skipgramCount in skipgrams.items():

    # Frequência de determinada palavra em determinado contexto
    # [#(w,c)]
    freqWordContext = skipgramCount

    # Frequência de determinada palavra em todos os contextos possíveis
    # [#(w)]
    freqWord = sumContexts[tokenWord]
    
    # Frequência de determinado contexto para todas as palavras possíveis
    # [#(c)]
    freqContext = sumWords[tokenContext]

    # Probabilidade de ocorrência de determinada palavra em determinado contexto
    # [P(w,c)]
    probWordContext = freqWordContext / numSkipgrams

    # Probabilidade de ocorrência de determinada palavra individualmente
    # [P(w)]
    probWord = freqWord / numSkipgrams

    # Probabilidade de ocorrência de determinado contexto individualmente
    # [P(c)]
    probContext = freqContext / numSkipgrams

    # Cálculo PPMI (Positive Pointwise Mutual Information)
    # [PPMI = max(0, log( P(w,c)/(P(w)P(c)) ))]
    PPMI = max(np.log2(probWordContext / (probWord * probContext)), 0)

    rowsIndex.append(tokenWord)
    columnsIndex.append(tokenContext)
    ppmiData.append(PPMI)

ppmiMatrix = sparse.csr_matrix((ppmiData, (rowsIndex, columnsIndex)))
ppmiMatrix

<99366x99366 sparse matrix of type '<class 'numpy.float64'>'
	with 3792872 stored elements in Compressed Sparse Row format>

### 7. Fatoração matricial usando SVD (Singular Value Decomposition)

In [19]:
from scipy.sparse.linalg import svds as SVD

# Dimensão proposta da matriz de valores singulares produzida pelo SVD
# [Hiperparâmetro]
embeddingSize = 50

U, D, V = SVD(ppmiMatrix, embeddingSize)

# Normalização das matrizes de vetores singulares produzidas pelo SVD
Unorm = U / np.sqrt(np.sum(U*U, axis=1, keepdims=True))
Vnorm = V / np.sqrt(np.sum(V*V, axis=1, keepdims=True))

# ???
wordVecs = Unorm

### 8. Visualização de palavras similares por similaridade por cosseno

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

# Cálculo dos 10 contextos mais similares a dada palavra utilizando a matriz de Word Embedding
def wordsSimilarity(word, matrix, n):
    wordIndex = token2index[word]

    # Resgate do vetor representante de determinada palavra
    if isinstance(matrix, sparse.csr_matrix):
        wordVec = matrix.getrow(wordIndex)
    else:
        wordVec = matrix[wordIndex:wordIndex+1, :]

    # Cálculo da similidade (similaridade de vetores por cosseno)
    similarity = cosine_similarity(matrix, wordVec).flatten()
    sortedIndexes = np.argsort(-similarity)

    # Retorno dos n contextos mais similares a dada palavra
    similarityContextScores = [(index2token[sortedIndex], similarity[sortedIndex]) 
                                for sortedIndex in sortedIndexes[:n+1] 
                                if index2token[sortedIndex] != word]

    return similarityContextScores

def wordSimilarityReport(word, matrix, n=5):
    print(f'\'{word}\'\t Frequência total: {unigrams[word]}', end='\n\t')

    similarityContextScores = wordsSimilarity(word, matrix, n)
    for context, similarity in similarityContextScores:
        print(f'(\'{context}\', {similarity})', end='\t')
    

In [45]:
examples = ['spotify', 'learning', 'deep', 'snapchat', 'facebook', 'musk', 'linux', 'safety']
for word in examples:
    wordSimilarityReport(word, wordVecs)
    print('\n'+'---'*20)

'spotify'	 Frequência total: 271
	('musixmatch', 0.7946992630661831)	('soundcloud', 0.7630237337321663)	('music', 0.7437292140911329)	('snapchat', 0.7258352530284516)	('beatport', 0.7068481903821319)	
------------------------------------------------------------
'learning'	 Frequência total: 3092
	('deep', 0.9679018143288161)	('machine', 0.9659037272340059)	('tensorlayer', 0.9351446662279084)	('selfstudying', 0.934205970281644)	('javacpp', 0.9335810170788844)	
------------------------------------------------------------
'deep'	 Frequência total: 1375
	('learning', 0.9679018143288162)	('advers', 0.9516899496215433)	('tinycnn', 0.9496797213952085)	('techniquesetc', 0.9483915875145995)	('javacpp', 0.9478161231761266)	
------------------------------------------------------------
'snapchat'	 Frequência total: 342
	('spotify', 0.7258352530284516)	('instagram', 0.7016543199091057)	('tinder', 0.6955864370596595)	('twitter', 0.6914984740807837)	('bitstrips', 0.6894812191440871)	
----------------

### 9. Visualizando as palavras através de um ScatterPlot interativo